In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import gc  # Garbage collector to free memory

# -----------------------------
# CONFIGURATIONS
# -----------------------------
OUTPUT_DIR = "dataset_large"       # Folder to save images + CSV
TOTAL_IMAGES = 5000               # Total dataset size (adjust as needed)
BATCH_SIZE = 1000                  # Number of images per batch
IMG_SIZE = (28, 28)                # Image resolution

os.makedirs(OUTPUT_DIR, exist_ok=True)
csv_path = os.path.join(OUTPUT_DIR, "tx_parameters.csv")

# Initialize CSV storage
if os.path.exists(csv_path):
    os.remove(csv_path)

# -----------------------------
# FUNCTION TO GENERATE WAVEFORM
# -----------------------------
def generate_waveform(frequency, alpha, beta, dielectric, noise_level=0.02):
    """Generate synthetic waveform based on transmission line parameters."""
    t = np.linspace(0, 1, 500)
    signal = np.sin(2 * np.pi * frequency * t + beta)
    signal *= np.exp(-alpha * t)
    noise = noise_level * np.random.randn(len(t))
    signal += noise
    signal = signal / np.max(np.abs(signal))  # Normalize
    return t, signal

# -----------------------------
# BATCH-WISE DATA GENERATION
# -----------------------------
print(f"Generating {TOTAL_IMAGES} waveform images in batches of {BATCH_SIZE}...")

for batch_start in range(0, TOTAL_IMAGES, BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, TOTAL_IMAGES)
    batch_data = []

    for i in tqdm(range(batch_start, batch_end), desc=f"Batch {batch_start // BATCH_SIZE + 1}"):
        # Random TX parameters
        frequency = np.random.uniform(1e6, 10e9)        # 1 MHz to 10 GHz
        alpha = np.random.uniform(0.01, 0.5)            # Attenuation
        beta = np.random.uniform(0.1, 2 * np.pi)        # Phase
        dielectric = np.random.uniform(1.5, 12.0)       # Permittivity

        # Generate waveform
        t, signal = generate_waveform(frequency, alpha, beta, dielectric)

        # Save image (faster & memory-safe)
        img_path = os.path.join(OUTPUT_DIR, f"waveform_{i}.png")
        plt.figure(figsize=(3, 3))
        plt.plot(t, signal, color='blue', linewidth=1.2)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(img_path, dpi=100, bbox_inches='tight', pad_inches=0)
        plt.close()

        # Store metadata
        batch_data.append([f"waveform_{i}.png", frequency, alpha, beta, dielectric])

    # Save batch metadata to CSV
    df = pd.DataFrame(batch_data, columns=["image", "frequency", "alpha", "beta", "dielectric"])
    df.to_csv(csv_path, mode="a", header=not os.path.exists(csv_path), index=False)

    # Free memory
    del batch_data, df
    gc.collect()

    print(f"✅ Saved batch {batch_start // BATCH_SIZE + 1} ({batch_end} images done)")

print(f"\n🎉 Dataset generation complete!")
print(f"📂 Images saved in: {OUTPUT_DIR}")
print(f"📄 Parameters CSV saved at: {csv_path}")


In [ ]:
# robust_cnn_regression.py
import os
import time
import joblib
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from sklearn.preprocessing import StandardScaler

# -------------------- CONFIG --------------------
DATA_DIR = "dataset_large"               # folder with images + tx_parameters.csv
CSV_PATH = os.path.join(DATA_DIR, "tx_parameters.csv")
MODEL_PATH = "waveform_cnn_best.pth"
SCALER_PATH = "label_scaler.pkl"

IMG_SIZE = 64
BATCH_SIZE = 32            # safe for RTX 3050 Ti
EPOCHS = 60
LR = 1e-4
PATIENCE = 8               # early stopping patience on val loss
CLIP_NORM = 1.0            # gradient clipping
VAL_RATIO = 0.1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_AMP = torch.cuda.is_available()  # use mixed precision only if GPU

print("Device:", DEVICE, "USE_AMP:", USE_AMP)

# -------------------- READ CSV & PREP LABEL SCALING --------------------
df = pd.read_csv(CSV_PATH)
# Expect CSV columns: [image, frequency, alpha, beta, dielectric]
assert df.shape[1] >= 5, "CSV must contain: filename, frequency, alpha, beta, dielectric"

# prepare lists
filenames = df.iloc[:, 0].astype(str).tolist()
labels_raw = df.iloc[:, 1:5].values.astype(float)  # shape (N,4)

# Transform frequency with log10 to stabilize scale (avoid log(0))
labels_trans = labels_raw.copy()
labels_trans[:, 0] = np.log10(np.maximum(labels_raw[:, 0], 1.0))  # log10(freq)

# Fit a StandardScaler on targets (fit on entire dataset then split; ok for synthetic)
label_scaler = StandardScaler()
labels_scaled_all = label_scaler.fit_transform(labels_trans)
joblib.dump(label_scaler, SCALER_PATH)
print("Saved label scaler to", SCALER_PATH)

# -------------------- TRAIN/VAL SPLIT --------------------
N = len(filenames)
indices = np.arange(N)
np.random.seed(42)
np.random.shuffle(indices)

val_count = int(N * VAL_RATIO)
val_idx = indices[:val_count]
train_idx = indices[val_count:]

train_files = [filenames[i] for i in train_idx]
val_files   = [filenames[i] for i in val_idx]
train_labels = labels_scaled_all[train_idx]
val_labels   = labels_scaled_all[val_idx]

print(f"Total samples: {N}, train: {len(train_files)}, val: {len(val_files)}")

# -------------------- DATASET CLASS --------------------
class WaveformImageDataset(Dataset):
    def __init__(self, file_list, labels_array, img_dir, transform=None):
        self.files = file_list
        self.labels = labels_array
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]
        fpath = os.path.join(self.img_dir, fname)
        image = Image.open(fpath).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label

# -------------------- TRANSFORMS & DATALOADERS --------------------
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),                         # -> [0,1]
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])  # -> [-1,1]
])

train_dataset = WaveformImageDataset(train_files, train_labels, DATA_DIR, transform=transform)
val_dataset   = WaveformImageDataset(val_files, val_labels, DATA_DIR, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=0, pin_memory=(DEVICE.type=="cuda"))
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=0, pin_memory=(DEVICE.type=="cuda"))

# -------------------- MODEL --------------------
class WaveformCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128,256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.AdaptiveAvgPool2d((4,4)),
            nn.Flatten(),
            nn.Linear(256*4*4, 512), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(512, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 4)
        )

    def forward(self, x):
        return self.net(x)

model = WaveformCNN().to(DEVICE)

# weight init
def init_weights(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
model.apply(init_weights)

# -------------------- LOSS, OPTIMIZER, SCHEDULER --------------------
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4, verbose=True)

scaler = None
if USE_AMP:
    from torch.amp import GradScaler, autocast
    scaler = GradScaler(device="cuda")

# -------------------- TRAIN + VALIDATE --------------------
best_val = float("inf")
best_epoch = -1
start_time = time.time()
no_improve = 0

for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss = 0.0
    n_train = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} [train]")
    for imgs, lbls in loop:
        imgs = imgs.to(DEVICE, non_blocking=True)
        lbls = lbls.to(DEVICE, non_blocking=True)

        optimizer.zero_grad()
        if USE_AMP:
            with autocast(device_type="cuda"):
                preds = model(imgs)
                loss = criterion(preds, lbls)
            scaler.scale(loss).backward()
            # gradient clipping
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_NORM)
            scaler.step(optimizer)
            scaler.update()
        else:
            preds = model(imgs)
            loss = criterion(preds, lbls)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_NORM)
            optimizer.step()

        train_loss += float(loss.item()) * imgs.size(0)
        n_train += imgs.size(0)
        loop.set_postfix(train_loss=train_loss / n_train)

    train_loss = train_loss / n_train

    # validation
    model.eval()
    val_loss = 0.0
    n_val = 0
    with torch.no_grad():
        for imgs, lbls in val_loader:
            imgs = imgs.to(DEVICE, non_blocking=True)
            lbls = lbls.to(DEVICE, non_blocking=True)
            if USE_AMP:
                with autocast(device_type="cuda"):
                    preds = model(imgs)
                    loss = criterion(preds, lbls)
            else:
                preds = model(imgs)
                loss = criterion(preds, lbls)
            val_loss += float(loss.item()) * imgs.size(0)
            n_val += imgs.size(0)
    val_loss = val_loss / n_val

    print(f"Epoch {epoch}: train_loss={train_loss:.6e}, val_loss={val_loss:.6e}")

    # scheduler step (ReduceLROnPlateau)
    scheduler.step(val_loss)

    # early stopping & save best
    if val_loss < best_val - 1e-12:
        best_val = val_loss
        best_epoch = epoch
        no_improve = 0
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"Saved best model (epoch {epoch}) val_loss={val_loss:.6e}")
    else:
        no_improve += 1
        print(f"No improvement count: {no_improve}/{PATIENCE}")

    if no_improve >= PATIENCE:
        print("Early stopping triggered.")
        break

total_time = time.time() - start_time
print(f"Training finished. Best epoch: {best_epoch}, best val loss: {best_val:.6e}, total_time={total_time:.1f}s")

# -------------------- HELPER: Prediction (loads scaler & model) --------------------
def load_model_and_scaler(model_path=MODEL_PATH, scaler_path=SCALER_PATH, device=DEVICE):
    m = WaveformCNN().to(device)
    m.load_state_dict(torch.load(model_path, map_location=device))
    m.eval()
    scl = joblib.load(scaler_path)
    return m, scl

def predict_image(image_path, model_obj, scaler_obj, img_size=IMG_SIZE, device=DEVICE):
    trans = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
    img = Image.open(image_path).convert("RGB")
    x = trans(img).unsqueeze(0).to(device)
    with torch.no_grad():
        if USE_AMP:
            from torch.amp import autocast
            with autocast(device_type="cuda"):
                pred_scaled = model_obj(x).cpu().numpy()
        else:
            pred_scaled = model_obj(x).cpu().numpy()
    # inverse scale and inverse log on frequency
    pred_unscaled = scaler_obj.inverse_transform(pred_scaled)
    pred_unscaled[:,0] = 10 ** pred_unscaled[:,0]  # inverse log10 frequency
    return pred_unscaled.flatten()

# Example usage after training:
# model_loaded, scaler_loaded = load_model_and_scaler()
# print(predict_image("dataset_large/waveform_0.png", model_loaded, scaler_loaded))


In [ ]:
# predict_compare.py
import os
import joblib
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import transforms

# ====== CONFIG (matches your files) ======
DATA_DIR    = "dataset_large"                    # folder containing images + csv
CSV_PATH    = os.path.join(DATA_DIR, "tx_parameters.csv")
MODEL_PATH  = "waveform_cnn_best.pth"            # model saved during training
SCALER_PATH = "label_scaler.pkl"                 # label scaler saved during training

IMG_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("DEVICE:", DEVICE)
# ==========================================


# ====== Model class — must match training exactly ======
class WaveformCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128,256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.AdaptiveAvgPool2d((4,4)),
            nn.Flatten(),
            nn.Linear(256*4*4, 512), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(512, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 4)
        )

    def forward(self, x):
        return self.net(x)
# =======================================================


# ====== Load model & scaler (robust) ======
def load_model_and_scaler(model_path=MODEL_PATH, scaler_path=SCALER_PATH, device=DEVICE):
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")
    if not os.path.exists(scaler_path):
        raise FileNotFoundError(f"Scaler file not found: {scaler_path}")

    model = WaveformCNN().to(device)
    # safe load: try torch.load with weights_only if available, else standard load
    try:
        # newer torch supports weights_only argument
        state = torch.load(model_path, map_location=device, weights_only=True)
    except TypeError:
        # fallback (older torch)
        state = torch.load(model_path, map_location=device)
    # If saved state was state_dict (most likely), load it
    if isinstance(state, dict):
        model.load_state_dict(state)
    else:
        # In case model was saved differently, try strict=False as fallback
        try:
            model.load_state_dict(state.state_dict())
        except Exception:
            model.load_state_dict(state, strict=False)

    model.eval()

    # load scaler (joblib)
    scaler = joblib.load(scaler_path)

    return model, scaler
# ==========================================


# ====== Image transform (match training) ======
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])
# ===========================================


# ====== Predict a single image: returns unscaled values (original units) ======
def predict_image(image_path, model, scaler, device=DEVICE):
    img = Image.open(image_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)          # shape (1,3,H,W)
    with torch.no_grad():
        out_scaled = model(x).cpu().numpy()            # shape (1,4)
    # inverse transform labels: scaler was fitted to [log10(freq), alpha, beta, dielectric]
    out_unscaled = scaler.inverse_transform(out_scaled)  # shape (1,4)
    # inverse log10 for frequency:
    out_unscaled[0,0] = 10 ** out_unscaled[0,0]
    return out_unscaled.flatten()   # [freq, alpha, beta, dielectric]
# ================================================================


# ====== Compare predictions for random samples from CSV ======
def predict_and_compare(n_samples=5, csv_path=CSV_PATH, img_dir=DATA_DIR):
    # load model + scaler
    model, scaler = load_model_and_scaler()

    # load CSV
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV not found: {csv_path}")
    df = pd.read_csv(csv_path)

    # ensure expected columns
    # first column should be image filename, next four = frequency, alpha, beta, dielectric
    # try to infer names gracefully
    cols = df.columns.tolist()
    if len(cols) < 5:
        raise ValueError("CSV must have at least 5 columns: image, frequency, alpha, beta, dielectric")
    img_col = cols[0]
    label_cols = cols[1:5]

    sample_df = df.sample(n=min(n_samples, len(df)), random_state=42).reset_index(drop=True)

    for idx, row in sample_df.iterrows():
        img_name = str(row[img_col])
        img_path = os.path.join(img_dir, img_name)
        if not os.path.exists(img_path):
            print(f"Warning: image not found, skipping: {img_path}")
            continue

        true_vals = row[label_cols].astype(float).values  # [freq, alpha, beta, dielectric]
        pred_vals = predict_image(img_path, model, scaler)

        # print numeric comparison
        print(f"\nSample {idx+1}: {img_name}")
        print("  Actual   -> Frequency: {:.6e}, Alpha: {:.6e}, Beta: {:.6e}, Dielectric: {:.6e}".format(*true_vals))
        print("  Predicted-> Frequency: {:.6e}, Alpha: {:.6e}, Beta: {:.6e}, Dielectric: {:.6e}".format(*pred_vals))

        # Plot: image + actual vs predicted (log y-scale so small values are visible)
        labels = ["Freq", "Alpha", "Beta", "Dielectric"]

        fig, axes = plt.subplots(1, 2, figsize=(12,4))
        # show image
        axes[0].imshow(Image.open(img_path).convert("RGB"))
        axes[0].axis("off")
        axes[0].set_title(img_name)

        # bar chart: use log scale on y
        x = np.arange(len(labels))
        axes[1].bar(x - 0.2, true_vals, width=0.4, label="Actual", color="tab:blue")
        axes[1].bar(x + 0.2, pred_vals, width=0.4, label="Pred", color="tab:orange", alpha=0.8)
        axes[1].set_xticks(x)
        axes[1].set_xticklabels(labels)
        axes[1].set_yscale("log")
        axes[1].set_title("Actual vs Predicted (log scale)")
        axes[1].legend()
        plt.tight_layout()
        plt.show()

# ================================================================


if __name__ == "__main__":
    # change n_samples if you want to display more/less
    predict_and_compare(n_samples=1)


In [ ]:
import os
import shutil
import pandas as pd

# Paths
LARGE_DIR = "dataset_large"
SAMPLE_DIR = "dataset_sample"
CSV_PATH = os.path.join(LARGE_DIR, "tx_parameters.csv")
SAMPLE_CSV_PATH = os.path.join(SAMPLE_DIR, "tx_parameters_sample.csv")

# Make sample folder
os.makedirs(SAMPLE_DIR, exist_ok=True)

# Load full CSV
df = pd.read_csv(CSV_PATH)

# Pick 10 samples (first 10, or you can shuffle for random)
sample_df = df.head(10)   # use df.sample(10, random_state=42) for random

# Copy images
for img_name in sample_df["image"]:
    src = os.path.join(LARGE_DIR, img_name)
    dst = os.path.join(SAMPLE_DIR, img_name)
    shutil.copy(src, dst)

# Save sample CSV
sample_df.to_csv(SAMPLE_CSV_PATH, index=False)

print(f"✅ 10 sample images + CSV saved in: {SAMPLE_DIR}")


In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch
import torch.nn as nn
from torchvision import transforms
import joblib

# -------------------- CONFIG --------------------
DATA_DIR = "dataset_large"
CSV_PATH = os.path.join(DATA_DIR, "tx_parameters.csv")
MODEL_PATH = "waveform_cnn_best.pth"
SCALER_PATH = "label_scaler.pkl"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 64
USE_AMP = torch.cuda.is_available()

print("Device:", DEVICE)

# -------------------- MODEL --------------------
class WaveformCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128,256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.AdaptiveAvgPool2d((4,4)),
            nn.Flatten(),
            nn.Linear(256*4*4, 512), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(512, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 4)
        )
    def forward(self, x):
        return self.net(x)

# -------------------- LOAD MODEL + SCALER --------------------
model = WaveformCNN().to(DEVICE)

# Safe load with weights_only=True (PyTorch 2.1+)
state_dict = torch.load(MODEL_PATH, map_location=DEVICE, weights_only=True)
model.load_state_dict(state_dict)
model.eval()

scaler = joblib.load(SCALER_PATH)

# -------------------- PREDICTION FUNCTION --------------------
def predict_image(image_path, model_obj=model, scaler_obj=scaler, img_size=IMG_SIZE, device=DEVICE):
    trans = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
    img = Image.open(image_path).convert("RGB")
    x = trans(img).unsqueeze(0).to(device)

    with torch.no_grad():
        if USE_AMP:
            from torch.amp import autocast
            with autocast(device_type="cuda"):
                pred_scaled = model_obj(x).cpu().numpy()
        else:
            pred_scaled = model_obj(x).cpu().numpy()

    # inverse scale
    pred_unscaled = scaler_obj.inverse_transform(pred_scaled)

    # fix overflow in frequency inverse log
    pred_unscaled[:,0] = 10 ** np.clip(pred_unscaled[:,0], 0, 12)

    return pred_unscaled.flatten()

# -------------------- EVALUATE ACCURACY --------------------
df = pd.read_csv(CSV_PATH)

best_idx = -1
best_name = None
best_err = float("inf")
best_gt = None
best_pred = None

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating"):
    img_name = row["image"]
    gt_values = row[["frequency", "alpha", "beta", "dielectric"]].values.astype(float)

    pred_values = predict_image(os.path.join(DATA_DIR, img_name))

    # safe denominator (avoid overflow / div by 0)
    denom = np.maximum(np.maximum(np.abs(gt_values), np.abs(pred_values)), 1e-9)
    rel_error = np.abs(pred_values - gt_values) / denom
    mean_error = rel_error.mean()

    if mean_error < best_err:
        best_err = mean_error
        best_idx = idx
        best_name = img_name
        best_gt = gt_values
        best_pred = pred_values

# -------------------- RESULT --------------------
print("\n🎯 Most Accurate Prediction Found:")
print(f"Image index: {best_idx}, Filename: {best_name}")
print(f"Ground Truth : {best_gt}")
print(f"Prediction   : {best_pred}")
print(f"Relative Error % per param: {(100* np.abs(best_pred - best_gt) / np.maximum(np.maximum(np.abs(best_gt), np.abs(best_pred)), 1e-9)).round(3)}")
print(f"Mean Relative Error %: {best_err*100:.3f}%")
